# BTC Spot Data Analysis
This notebook fetches and analyzes BTC data from various exchanges.

In [3]:
from dotenv import load_dotenv
load_dotenv()

False

## Exchange Fetchers
Classes to fetch OHLCV data from different exchanges.

In [5]:
# ======================================================================================
# Fetchers por exchange (1D)
# ======================================================================================

class HyperliquidFetcher:
    """Hyperliquid candles via native Info API (candleSnapshot)."""
    ENDPOINTS = [
        "https://api.hyperliquid.xyz/info",
        "https://api-ui.hyperliquid.xyz/info",
    ]

    @staticmethod
    def _post(payload: dict) -> Optional[dict]:
        last_err = None
        for url in HyperliquidFetcher.ENDPOINTS:
            try:
                r = requests.post(url, json=payload, headers=USER_AGENT, timeout=30)
                if r.status_code == 200:
                    return r.json()
                last_err = f"HTTP {r.status_code}: {r.text[:120]}"
            except Exception as e:
                last_err = str(e)
        print(f"[Hyperliquid] API error: {last_err}")
        return None

    @staticmethod
    def fetch(symbol: str = "BTC/USDC", interval: str = "1d", start_ms: Optional[int] = None) -> pd.DataFrame:
        coin = symbol.split('/')[0].upper()
        since = start_ms or to_ms(datetime(2019, 9, 1, tzinfo=timezone.utc))
        end_ms = to_ms(datetime.now(timezone.utc))
        print(f"[Hyperliquid] Fetch {symbol} interval={interval} since={ms_to_utc(since).date()} → {ms_to_utc(end_ms).date()}")
        payload = {
            "type": "candleSnapshot",
            "req": {"coin": coin, "interval": interval, "startTime": since, "endTime": end_ms},
        }
        js = HyperliquidFetcher._post(payload)
        if not js or not isinstance(js, list) or not js:
            print("[Hyperliquid] No data returned")
            return pd.DataFrame()
        rows = []
        for k in js:
            t = int(k.get("t", 0))
            o = float(k.get("o", 0)); h = float(k.get("h", 0)); l = float(k.get("l", 0)); c = float(k.get("c", 0))
            v = float(k.get("v", 0))
            rows.append([t, o, h, l, c, v])
        df = normalize_ohlcv_rows(rows, symbol, "Hyperliquid")
        if start_ms is not None and ONLY_LAST_2_DAYS_IF_MISSING:
            start_dt = pd.to_datetime(start_ms, unit='ms', utc=True)
            df = df[df["timestamp"] >= start_dt]
        print(f"[Hyperliquid] Fetched rows: {len(df)}; range: {df['timestamp'].min().date() if not df.empty else '-'} → {df['timestamp'].max().date() if not df.empty else '-'}")
        return df


class BinanceFetcher:
    BASE = "https://api.binance.com/api/v3/klines"

    @staticmethod
    def fetch(symbol: str = "BTC/USDT", interval: str = "1d", start_ms: Optional[int] = None) -> pd.DataFrame:
        sym = symbol.replace("/", "")  # BTCUSDT
        start = start_ms or to_ms(datetime(2019,9,1,tzinfo=timezone.utc))
        all_rows: List[List] = []
        while True:
            params = {"symbol": sym, "interval": interval, "limit": 1000, "startTime": start}
            r = requests.get(BinanceFetcher.BASE, params=params, headers=USER_AGENT, timeout=30)
            if r.status_code != 200:
                print("[Binance] HTTP", r.status_code, r.text[:120]); break
            batch = r.json()
            if not batch:
                break
            for k in batch:
                all_rows.append([k[0], k[1], k[2], k[3], k[4], k[5]])
            if len(batch) < 1000:
                break
            start = batch[-1][0] + 1
            time.sleep(0.12)
        return normalize_ohlcv_rows(all_rows, symbol, "Binance")


class BybitFetcher:
    BASE = "https://api.bybit.com/v5/market/kline"

    @staticmethod
    def fetch(symbol: str = "BTC/USDT", interval: str = "D", start_ms: Optional[int] = None) -> pd.DataFrame:
        all_rows: List[List] = []
        end = to_ms(datetime.now(timezone.utc))
        start = start_ms
        while True:
            params = {"category":"spot","symbol":symbol.replace("/",""),"interval":interval,"limit":1000,"end":end}
            if start is not None:
                params["start"] = start
            r = requests.get(BybitFetcher.BASE, params=params, headers=USER_AGENT, timeout=30)
            if r.status_code != 200:
                print("[Bybit] HTTP", r.status_code, r.text[:120]); break
            js = r.json()
            if js.get("retCode", -1) != 0:
                print("[Bybit] API", js.get("retMsg")); break
            lst = js.get("result",{}).get("list",[])
            if not lst:
                break
            for k in lst:
                all_rows.append([int(k[0]), k[1], k[2], k[3], k[4], k[5]])
            if len(lst) < 1000:
                break
            end = int(lst[-1][0]) - 1
            time.sleep(0.12)
        return normalize_ohlcv_rows(all_rows, symbol, "Bybit")


class OKXFetcher:
    BASE = "https://www.okx.com/api/v5/market/history-candles"
    BASE_REGULAR = "https://www.okx.com/api/v5/market/candles"  # Para datos recientes

    @staticmethod
    def fetch(symbol: str = "BTC/USDT", interval: str = "1D", start_ms: Optional[int] = None) -> pd.DataFrame:
        """
        Fetch OHLCV data from OKX with proper pagination and recent data handling.
        Uses both history-candles (for historical) and regular candles (for recent) endpoints.
        """
        okx_symbol = symbol.replace("/", "-")
        limit = 100
        all_rows: List[List] = []
        seen_ts = set()
        
        print(f"[OKX] Fetching {symbol} interval={interval}")
        
        # PASO 1: Obtener datos recientes usando el endpoint regular (últimos 100 días)
        print(f"[OKX] Step 1: Fetching recent data for {symbol}...")
        params_recent = {"instId": okx_symbol, "bar": interval, "limit": limit}
        
        try:
            r = requests.get(OKXFetcher.BASE_REGULAR, params=params_recent, headers=USER_AGENT, timeout=30)
            if r.status_code == 200:
                js = r.json()
                if js.get("code") == "0":
                    recent_data = js.get("data", [])
                    print(f"[OKX] Got {len(recent_data)} recent candles")
                    for k in recent_data:
                        t = int(k[0])
                        if t not in seen_ts:
                            seen_ts.add(t)
                            all_rows.append([t, k[1], k[2], k[3], k[4], k[5]])
                    
                    if recent_data:
                        oldest_recent = min(int(k[0]) for k in recent_data)
                        newest_recent = max(int(k[0]) for k in recent_data)
                        print(f"[OKX] Recent data range: {ms_to_utc(oldest_recent).date()} → {ms_to_utc(newest_recent).date()}")
                else:
                    print(f"[OKX] Recent data error: {js.get('msg')}")
        except Exception as e:
            print(f"[OKX] Exception fetching recent data: {e}")
        
        # PASO 2: Obtener datos históricos usando history-candles
        print(f"[OKX] Step 2: Fetching historical data for {symbol}...")
        
        # Verificar primero si hay datos históricos disponibles
        test_params = {"instId": okx_symbol, "bar": interval, "limit": 1}
        try:
            test_r = requests.get(OKXFetcher.BASE, params=test_params, headers=USER_AGENT, timeout=30)
            if test_r.status_code == 200:
                test_js = test_r.json()
                if test_js.get("code") == "0":
                    test_data = test_js.get("data", [])
                    if not test_data:
                        print(f"[OKX] WARNING: No historical data available for {symbol}")
                        print(f"[OKX] This might be a new or low-liquidity pair on OKX")
                    else:
                        # Si hay datos históricos, proceder con la paginación
                        pages = 0
                        max_pages = 50
                        after_ts = None
                        
                        while pages < max_pages:
                            params = {"instId": okx_symbol, "bar": interval, "limit": limit}
                            if after_ts:
                                params["after"] = after_ts
                            
                            r = requests.get(OKXFetcher.BASE, params=params, headers=USER_AGENT, timeout=30)
                            if r.status_code != 200:
                                print(f"[OKX] HTTP {r.status_code}")
                                break
                            
                            js = r.json()
                            if js.get("code") != "0":
                                error_msg = js.get("msg", "Unknown error")
                                if "Invalid" in error_msg or "not exist" in error_msg:
                                    print(f"[OKX] Pair {okx_symbol} might not be available for historical data")
                                else:
                                    print(f"[OKX] API error: {error_msg}")
                                break
                            
                            data = js.get("data", [])
                            if not data:
                                break
                            
                            added = 0
                            oldest_in_batch = None
                            
                            for k in data:
                                t = int(k[0])
                                if t not in seen_ts:
                                    seen_ts.add(t)
                                    all_rows.append([t, k[1], k[2], k[3], k[4], k[5]])
                                    added += 1
                                    if oldest_in_batch is None or t < oldest_in_batch:
                                        oldest_in_batch = t
                            
                            if added == 0:
                                print(f"[OKX] No new historical data, stopping at page {pages}")
                                break
                            
                            if oldest_in_batch:
                                after_ts = oldest_in_batch
                                if pages % 5 == 0 and pages > 0:
                                    print(f"[OKX] Historical: {pages} pages, {len(all_rows)} total candles, "
                                          f"oldest so far: {ms_to_utc(oldest_in_batch).date()}")
                            
                            # Check if we've reached the start threshold
                            if start_ms and oldest_in_batch and oldest_in_batch <= start_ms:
                                print(f"[OKX] Reached start threshold")
                                break
                            
                            pages += 1
                            time.sleep(0.15)
                        
                        if pages > 0:
                            print(f"[OKX] Historical fetch complete: {pages} pages")
                else:
                    print(f"[OKX] Test query failed: {test_js.get('msg')}")
        except Exception as e:
            print(f"[OKX] Exception during historical fetch: {e}")
        
        # PASO 3: Filtrar y procesar datos
        if start_ms:
            all_rows = [r for r in all_rows if r[0] >= start_ms]
        
        # Ordenar por timestamp
        all_rows.sort(key=lambda x: x[0])
        
        df = normalize_ohlcv_rows(all_rows, symbol, "OKX")
        
        if not df.empty:
            print(f"[OKX] {symbol} final dataset: {len(df)} rows")
            print(f"[OKX] Date range: {df['timestamp'].min().date()} → {df['timestamp'].max().date()}")
            
            # Verificar gaps significativos
            if len(df) > 1:
                df_sorted = df.sort_values('timestamp')
                date_diffs = df_sorted['timestamp'].diff()
                max_gap = date_diffs.max()
                if max_gap > pd.Timedelta(days=30):
                    gap_idx = date_diffs.idxmax()
                    print(f"[OKX] WARNING: Large gap detected ({max_gap.days} days) "
                          f"at {df_sorted.loc[gap_idx, 'timestamp'].date()}")
        else:
            print(f"[OKX] WARNING: No data retrieved for {symbol}")
        
        # DIAGNÓSTICO ESPECIAL PARA PARES CON POCOS DATOS
        if not df.empty and len(df) < 100:
            print(f"[OKX] ⚠️ LIMITED DATA for {symbol}: Only {len(df)} candles available")
            print(f"[OKX] This suggests {symbol} is:")
            print(f"[OKX]   - A newly listed pair on OKX")
            print(f"[OKX]   - A low liquidity/volume pair")
            print(f"[OKX]   - Or has limited historical data on this exchange")
            print(f"[OKX] Consider using a different exchange for this pair if more history is needed")
        
        return df


class CoinbaseFetcher:
    BASE = "https://api.exchange.coinbase.com/products/{}/candles"

    @staticmethod
    def _iso_z(dt: datetime) -> str:
        return dt.astimezone(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')

    @staticmethod
    def _pick_product(symbol: str) -> str:
        if symbol in ("BTC/USD", "BTC-USD"): return "BTC-USD"
        if symbol in ("BTC/USDT", "BTC-USDT", "BTC/USDC", "BTC-USDC"): return "BTC-USD"
        return symbol.replace("/","-")

    @staticmethod
    def fetch(symbol: str = "BTC/USD", granularity_sec: int = 86400, start_ms: Optional[int] = None,
              include_partial_today: bool = True) -> pd.DataFrame:
        product = CoinbaseFetcher._pick_product(symbol)
        end_dt = datetime.now(timezone.utc)
        since_dt = datetime(2019,9,1,tzinfo=timezone.utc) if not start_ms else datetime.fromtimestamp(start_ms/1000, tz=timezone.utc)
        step = granularity_sec * 300
        cur_end = end_dt
        all_rows: List[List] = []
        while cur_end > since_dt:
            cur_start = max(since_dt, cur_end - timedelta(seconds=step - granularity_sec))
            params = {"granularity": granularity_sec, "start": CoinbaseFetcher._iso_z(cur_start), "end": CoinbaseFetcher._iso_z(cur_end)}
            r = requests.get(CoinbaseFetcher.BASE.format(product), params=params, headers=USER_AGENT, timeout=30)
            if r.status_code == 404:
                break
            if r.status_code != 200:
                print("[Coinbase] HTTP", r.status_code, r.text[:120]); break
            batch = r.json()  # [time, low, high, open, close, volume] DESC
            if not batch:
                break
            for k in batch:
                all_rows.append([k[0]*1000, k[3], k[2], k[1], k[4], k[5]])
            oldest = min(b[0] for b in batch)
            cur_end = datetime.fromtimestamp(oldest - granularity_sec, tz=timezone.utc)
            time.sleep(0.12)
        all_rows.sort(key=lambda r: r[0])
        df = normalize_ohlcv_rows(all_rows, symbol, "Coinbase")
        if include_partial_today and granularity_sec == 86400:
            if not df.empty and df["timestamp"].dt.date.max() < datetime.now(timezone.utc).date():
                start_h = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
                params = {"granularity": 3600, "start": CoinbaseFetcher._iso_z(start_h), "end": CoinbaseFetcher._iso_z(end_dt)}
                r = requests.get(CoinbaseFetcher.BASE.format(product), params=params, headers=USER_AGENT, timeout=30)
                if r.status_code == 200:
                    h = r.json()
                    if h:
                        h.sort(key=lambda x: x[0])
                        o = float(h[0][3]); hhi = float(max(x[2] for x in h)); hlo = float(min(x[1] for x in h)); c = float(h[-1][4]); v = float(sum(x[5] for x in h))
                        row = [to_ms(start_h), o, hhi, hlo, c, v]
                        df = pd.concat([df, normalize_ohlcv_rows([row], symbol, "Coinbase")], ignore_index=True)
        return df


class CryptoComFetcher:
    BASE = "https://api.crypto.com/exchange/v1/public/get-candlestick"

    @staticmethod
    def fetch(symbol: str = "BTC/USDT", interval: str = "1D", start_ms: Optional[int] = None) -> pd.DataFrame:
        inst = symbol.replace("/", "_")
        all_rows: List[List] = []
        end_ts = to_ms(datetime.now(timezone.utc))
        since = start_ms or to_ms(datetime(2019,9,1,tzinfo=timezone.utc))
        while end_ts > since:
            params = {"instrument_name": inst, "timeframe": interval, "count": 300, "end_ts": end_ts}
            r = requests.get(CryptoComFetcher.BASE, params=params, headers=USER_AGENT, timeout=30)
            if r.status_code != 200:
                print("[Crypto.com] HTTP", r.status_code, r.text[:120]); break
            js = r.json()
            if js.get("code") != 0:
                print("[Crypto.com] API", js.get("message")); break
            data = js.get("result",{}).get("data",[])
            if not data:
                break
            for k in data:
                all_rows.append([k['t'], k['o'], k['h'], k['l'], k['c'], k.get('v',0)])
            oldest = min(k['t'] for k in data)
            end_ts = oldest - 1
            time.sleep(0.1)
        return normalize_ohlcv_rows(all_rows, symbol, "Crypto.com")


class UpbitFetcher:
    BASE = "https://api.upbit.com/v1"
    
    @staticmethod
    def fetch_days(symbol: str = "BTC/KRW", start_ms: Optional[int] = None) -> pd.DataFrame:
        """
        Fetch daily candles from Upbit with robust date handling.
        Fixed timezone issues and deprecated datetime methods.
        """
        # Determine market based on symbol
        if symbol.upper().endswith("KRW"):
            market = "KRW-BTC"
        elif symbol.upper().endswith("USDT"):
            market = "USDT-BTC"
        else:
            market = "KRW-BTC"  # Default to KRW
        
        url = UpbitFetcher.BASE + "/candles/days"
        count = 200
        
        # Use timezone-aware datetime (Python 3.12+ compatible)
        to_dt = datetime.now(timezone.utc)
        threshold_dt = datetime(2017, 1, 1, tzinfo=timezone.utc)
        since_dt = datetime.fromtimestamp(start_ms/1000, tz=timezone.utc) if start_ms else None
        
        all_rows: List[List] = []
        pages = 0
        max_pages = 30  # Safety limit
        
        print(f"[Upbit] Fetching {symbol} (market: {market})...")
        
        while pages < max_pages:
            try:
                # Format datetime for Upbit API (ISO 8601)
                to_str = to_dt.strftime("%Y-%m-%dT%H:%M:%S")
                params = {
                    "market": market,
                    "count": count,
                    "to": to_str
                }
                
                # Make request
                r = requests.get(url, params=params, headers=USER_AGENT, timeout=30)
                
                if r.status_code == 429:
                    print("[Upbit] Rate limit hit, waiting 1 second...")
                    time.sleep(1)
                    continue
                
                if r.status_code != 200:
                    print(f"[Upbit] HTTP {r.status_code}: {r.text[:200]}")
                    break
                
                batch = r.json()
                
                # Check if response is an error dict
                if isinstance(batch, dict) and 'error' in batch:
                    print(f"[Upbit] API error: {batch.get('error', {}).get('message', 'Unknown error')}")
                    break
                
                if not batch or not isinstance(batch, list):
                    print(f"[Upbit] No more data or unexpected response type")
                    break
                
                # Process batch
                batch_count = 0
                oldest_dt_in_batch = None
                
                for k in batch:
                    try:
                        # Get UTC timestamp
                        iso = k.get("candle_date_time_utc")
                        if not iso:
                            continue
                        
                        # Parse ISO timestamp robustly
                        if 'T' in iso:
                            if iso.endswith('Z'):
                                dt = datetime.fromisoformat(iso[:-1] + '+00:00')
                            elif '+' in iso or '-' in iso[-6:]:
                                dt = datetime.fromisoformat(iso)
                            else:
                                dt = datetime.fromisoformat(iso).replace(tzinfo=timezone.utc)
                        else:
                            dt = pd.to_datetime(iso, utc=True).to_pydatetime()
                        
                        if dt.tzinfo is None:
                            dt = dt.replace(tzinfo=timezone.utc)
                        
                        if oldest_dt_in_batch is None or dt < oldest_dt_in_batch:
                            oldest_dt_in_batch = dt
                        
                        ts_ms = int(dt.timestamp() * 1000)
                        
                        o = float(k.get("opening_price", 0))
                        h = float(k.get("high_price", 0))
                        l = float(k.get("low_price", 0))
                        c = float(k.get("trade_price", 0))
                        v = float(k.get("candle_acc_trade_volume", 0))  # Volume in base currency (BTC)
                        
                        all_rows.append([ts_ms, o, h, l, c, v])
                        batch_count += 1
                        
                    except (ValueError, KeyError, TypeError) as e:
                        print(f"[Upbit] Error parsing candle: {e}")
                        continue
                
                if batch_count == 0:
                    print(f"[Upbit] No valid candles in batch, stopping")
                    break
                
                print(f"[Upbit] Page {pages+1}: {batch_count} candles, oldest: {oldest_dt_in_batch.date() if oldest_dt_in_batch else 'N/A'}")
                
                if oldest_dt_in_batch:
                    if since_dt and oldest_dt_in_batch <= since_dt:
                        print(f"[Upbit] Reached start date threshold")
                        break
                    
                    if oldest_dt_in_batch <= threshold_dt:
                        print(f"[Upbit] Reached historical threshold (2017)")
                        break
                    
                    to_dt = oldest_dt_in_batch - timedelta(seconds=1)
                else:
                    print(f"[Upbit] Could not determine oldest date in batch, stopping")
                    break
                
                if len(batch) < count:
                    print(f"[Upbit] Got {len(batch)} < {count} candles, reached end of data")
                    break
                
                pages += 1
                time.sleep(0.15)  # Rate limiting
                
            except requests.exceptions.RequestException as e:
                print(f"[Upbit] Request error: {e}")
                break
            except Exception as e:
                print(f"[Upbit] Unexpected error: {e}")
                break
        
        df = normalize_ohlcv_rows(all_rows, symbol, "Upbit")
        
        if start_ms is not None and ONLY_LAST_2_DAYS_IF_MISSING:
            start_dt = pd.to_datetime(start_ms, unit='ms', utc=True)
            df = df[df["timestamp"] >= start_dt]
            print(f"[Upbit] Filtered to dates >= {start_dt.date()}")
        
        if not df.empty:
            print(f"[Upbit] Final dataset: {len(df)} rows, range: {df['timestamp'].min().date()} → {df['timestamp'].max().date()}")
        else:
            print(f"[Upbit] No data retrieved")
        
        return df

def test_upbit_api():
    """
    Test Upbit API connectivity and response format
    """
    print("\n[Upbit Test] Testing API connectivity...")
    
    # Test markets endpoint first
    try:
        markets_url = "https://api.upbit.com/v1/market/all"
        r = requests.get(markets_url, timeout=10)
        if r.status_code == 200:
            markets = r.json()
            btc_markets = [m for m in markets if 'BTC' in m.get('market', '')]
            print(f"[Upbit Test] ✓ Markets endpoint OK, found {len(btc_markets)} BTC markets")
            
            for m in btc_markets[:5]:
                print(f"  - {m['market']}: {m.get('korean_name', m.get('english_name', 'N/A'))}")
        else:
            print(f"[Upbit Test] ✗ Markets endpoint failed: {r.status_code}")
    except Exception as e:
        print(f"[Upbit Test] ✗ Markets test failed: {e}")
    
    # Test candles endpoint
    try:
        candles_url = "https://api.upbit.com/v1/candles/days"
        params = {"market": "KRW-BTC", "count": 1}
        r = requests.get(candles_url, params=params, timeout=10)
        
        if r.status_code == 200:
            data = r.json()
            if data and isinstance(data, list) and len(data) > 0:
                candle = data[0]
                print(f"[Upbit Test] ✓ Candles endpoint OK")
                print(f"  Latest candle date: {candle.get('candle_date_time_utc')}")
                print(f"  Price: {candle.get('trade_price'):,.0f} KRW")
                print(f"  Volume: {candle.get('candle_acc_trade_volume'):.4f} BTC")
            else:
                print(f"[Upbit Test] ✗ Unexpected response format: {data}")
        else:
            print(f"[Upbit Test] ✗ Candles endpoint failed: {r.status_code}")
            print(f"  Response: {r.text[:200]}")
    except Exception as e:
        print(f"[Upbit Test] ✗ Candles test failed: {e}")
    
    print("[Upbit Test] Complete\n")


class BitgetFetcher:
    """Bitget spot history candles (v2). Paginación por endTime (limit<=200)."""
    BASE = "https://api.bitget.com/api/v2/spot/market/history-candles"

    @staticmethod
    def _map_granularity(interval: str) -> str:
        s = (interval or "1D").strip().lower()
        if s in ("1d", "1day", "1dutc"): return "1Dutc"
        if s in ("3d", "3day", "3dutc"): return "3Dutc"
        if s in ("1w", "1week", "1wutc"): return "1Wutc"
        if s == "1m": return "1min"
        if s == "3m": return "3min"
        if s == "5m": return "5min"
        if s == "15m": return "15min"
        if s == "30m": return "30min"
        if s == "1h": return "1h"
        if s == "4h": return "4h"
        if s == "6h": return "6h"
        if s == "12h": return "12h"
        if s in ("1mth", "1month"): return "1M"
        return interval

    @staticmethod
    def fetch(symbol: str = "BTC/USDT", interval: str = "1D", start_ms: Optional[int] = None) -> pd.DataFrame:
        sym = symbol.replace("/", "")
        end = to_ms(datetime.now(timezone.utc))
        all_rows: List[List] = []
        limit = 200
        gran = BitgetFetcher._map_granularity(interval)
        prev_oldest: Optional[int] = None

        while True:
            params = {"symbol": sym, "granularity": gran, "endTime": end, "limit": limit}
            r = requests.get(BitgetFetcher.BASE, params=params, headers=USER_AGENT, timeout=30)
            if r.status_code != 200:
                print("[Bitget] HTTP", r.status_code, r.text[:160])
                break
            js = r.json()
            if js.get("code") != "00000":
                print("[Bitget] API", js.get("msg"))
                break
            data = js.get("data", [])
            if not data:
                break

            for k in data:
                try:
                    t = int(k[0]); o = float(k[1]); h = float(k[2]); l = float(k[3]); c = float(k[4]); v = float(k[5])
                    all_rows.append([t, o, h, l, c, v])
                except Exception:
                    continue

            oldest = min(int(k[0]) for k in data)
            if start_ms and oldest <= start_ms:
                break
            if prev_oldest is not None and oldest >= prev_oldest:
                break  # sin progreso
            prev_oldest = oldest
            end = oldest - 1
            time.sleep(0.12)

        df = normalize_ohlcv_rows(all_rows, symbol, "Bitget")
        if start_ms is not None:
            df = df[df["timestamp"] >= pd.to_datetime(start_ms, unit="ms", utc=True)]
        print(f"[Bitget] Fetched rows: {len(df)}; range: "
              f"{df['timestamp'].min().date() if not df.empty else '-'} → "
              f"{df['timestamp'].max().date() if not df.empty else '-'}")
        return df


class MEXCFetcher:
    BASE = "https://api.mexc.com/api/v3/klines"
    @staticmethod
    def fetch(symbol="BTC/USDT", interval="1d", start_ms=None) -> pd.DataFrame:
        sym = symbol.replace("/", "")
        end = to_ms(datetime.now(timezone.utc))
        all_rows = []
        limit = 1000
        prev_oldest = None
        while True:
            params = {"symbol": sym, "interval": interval, "limit": limit, "endTime": end}
            r = requests.get(MEXCFetcher.BASE, params=params, headers=USER_AGENT, timeout=30)
            if r.status_code != 200:
                print("[MEXC] HTTP", r.status_code, r.text[:120]); break
            batch = r.json()
            if not batch:
                break
            for k in batch:
                all_rows.append([int(k[0]), float(k[1]), float(k[2]), float(k[3]), float(k[4]), float(k[5])])
            oldest = min(int(k[0]) for k in batch)
            if start_ms and oldest <= start_ms:
                break
            if prev_oldest is not None and oldest >= prev_oldest:
                break
            prev_oldest = oldest
            end = oldest - 1
            time.sleep(0.12)
        df = normalize_ohlcv_rows(all_rows, symbol, "MEXC")
        if start_ms is not None and ONLY_LAST_2_DAYS_IF_MISSING:
            df = df[df["timestamp"] >= pd.to_datetime(start_ms, unit="ms", utc=True)]
        return df


## FX and Premium Metrics
Functions to handle currency conversion and premium calculations (Coinbase, Kimchi).

In [6]:
# ======================================================================================
# FX USDKRW (fallbacks sin API key) y métricas
# ======================================================================================

def fetch_usdkrw_timeseries(start_date: str, end_date: str) -> pd.DataFrame:
    """Obtains USDKRW (KRW por USD) para [start_date, end_date].
    Fallbacks: Frankfurter (app/dev) → FRED CSV (EXKOUS). Normalizes orientation (<1 → invertido).
    """
    used = None
    out: Optional[pd.DataFrame] = None
    for base_url in ("https://api.frankfurter.app", "https://api.frankfurter.dev"):
        try:
            url = f"{base_url}/{start_date}..{end_date}"
            params = {"from": "USD", "to": "KRW"}
            r = requests.get(url, params=params, headers=USER_AGENT, timeout=30)
            if r.status_code == 200:
                js = r.json(); rates = js.get("rates", {})
                if rates:
                    rows = [{"date": pd.to_datetime(d, utc=True), "USDKRW": float(rec["KRW"])} for d, rec in sorted(rates.items())]
                    out = pd.DataFrame(rows).set_index("date").sort_index()
                    used = f"Frankfurter {base_url}"
                    break
        except Exception:
            pass
    if out is None:
        fred_url = "https://fred.stlouisfed.org/graph/fredgraph.csv"
        try:
            r = requests.get(fred_url, params={"id": "EXKOUS"}, headers=USER_AGENT, timeout=30)
            r.raise_for_status()
            csv_df = pd.read_csv(io.StringIO(r.text))
            csv_df.rename(columns={"DATE": "date", "EXKOUS": "USDKRW"}, inplace=True)
            csv_df["date"] = pd.to_datetime(csv_df["date"], utc=True)
            csv_df["USDKRW"] = pd.to_numeric(csv_df["USDKRW"], errors="coerce")
            csv_df = csv_df.dropna().set_index("date").sort_index()
            mask = (csv_df.index.date >= pd.to_datetime(start_date).date()) & (csv_df.index.date <= pd.to_datetime(end_date).date())
            out = csv_df.loc[mask]
            used = "FRED EXKOUS"
        except Exception:
            pass
    if out is None or out.empty:
        raise RuntimeError("Could not obtain USDKRW desde las fuentes públicas (Frankfurter/FRED)")
    med = out["USDKRW"].median()
    if pd.notna(med) and med < 1:
        out["USDKRW"] = 1.0 / out["USDKRW"]
        used += " (invertido)"
    print(f"[FX] USDKRW fuente: {used}; rango: {out.index.min().date()} → {out.index.max().date()} (mediana={out['USDKRW'].median():.2f})")
    return out


def compute_coinbase_premium(cb_df: pd.DataFrame, usd_dfs: List[pd.DataFrame]) -> pd.DataFrame:
    """Coinbase premium using weights en USD (close*volume). Aplica penalización a Hyperliquid."""
    cb_dates = ensure_daily(cb_df)[["date","close","volume"]].rename(columns={"close":"cb_close","volume":"cb_vol"})

    ref_parts = []
    for d in usd_dfs:
        if d is None or d.empty:
            continue
        dd = ensure_daily(d)[["date","close","volume","exchange"]].copy()
        dd["w_usd"] = dd["close"] * dd["volume"]
        dd.loc[dd["exchange"].str.contains("Hyperliquid", case=False, na=False), "w_usd"] *= HYPERLIQUID_WEIGHT_PENALTY
        ref_parts.append(dd[["date","close","w_usd"]])
    if not ref_parts:
        return pd.DataFrame()

    ref = pd.concat(ref_parts, ignore_index=True)
    ref = ref.groupby("date").apply(lambda g: pd.Series({
        "ref_close": (g["close"] * g["w_usd"]).sum() / max(g["w_usd"].sum(), 1e-12)
    })).reset_index()

    m = cb_dates.merge(ref, on="date", how="inner")
    m["coinbase_premium_pct"] = (m["cb_close"] - m["ref_close"]) / m["ref_close"] * 100
    return m[["date","cb_close","ref_close","coinbase_premium_pct"]]


def compute_kimchi_premium(upbit_krw_df: pd.DataFrame, ref_usd_df: pd.DataFrame, fx_df: pd.DataFrame) -> pd.DataFrame:
    U = ensure_daily(upbit_krw_df)[["date","close"]].rename(columns={"close":"upbit_close_krw"})
    R = ensure_daily(ref_usd_df)[["date","close"]].rename(columns={"close":"ref_close_usd"})
    F = fx_df.reset_index().rename(columns={"date":"date"})
    m = U.merge(R, on="date", how="inner").merge(F, on="date", how="left").sort_values("date")
    m["USDKRW"] = m["USDKRW"].ffill().bfill()
    m["upbit_close_usd"] = m["upbit_close_krw"] / m["USDKRW"]
    m["kimchi_pct"] = (m["upbit_close_usd"] - m["ref_close_usd"]) / m["ref_close_usd"] * 100
    return m[["date","upbit_close_krw","USDKRW","upbit_close_usd","ref_close_usd","kimchi_pct"]]


## Aggregation and Visualization
Aggregating data across exchanges and generating charts.

In [7]:
# ======================================================================================
# Agregado y gráficos
# ======================================================================================

def aggregate_usd_candles(dfs: List[pd.DataFrame], outlier_factor: float = 0.35) -> pd.DataFrame:
    """Robust daily aggregation (open/close VWAP; high=max, low=min) con filtro de outliers.
    Pesos en USD = close*volume; aplica penalización a Hyperliquid.
    """
    parts = []
    for d in dfs:
        if d is None or d.empty:
            continue
        dd = ensure_daily(d)
        if "exchange" not in dd.columns:
            dd["exchange"] = d.get("exchange", "?")
        parts.append(dd[["date","open","high","low","close","volume","exchange"]])
    if not parts:
        return pd.DataFrame()
    big = pd.concat(parts, ignore_index=True)
    big = big.dropna(subset=["open","high","low","close","volume"]).copy()
    big = big[(big["close"] > 0) & (big["open"] > 0) & (big["high"] > 0) & (big["low"] > 0) & (big["volume"] > 0)]

    med = big.groupby("date")["close"].median().rename("med_close")
    big = big.merge(med, on="date", how="left")
    lo, hi = (1.0 - outlier_factor), (1.0 + outlier_factor)
    mask = (big["open"]>=lo*big["med_close"])&(big["open"]<=hi*big["med_close"])&\
           (big["high"]<=hi*big["med_close"])&(big["low"]>=lo*big["med_close"])&\
           (big["close"]>=lo*big["med_close"])&(big["close"]<=hi*big["med_close"]) 
    dropped = int((~mask).sum())
    if dropped > 0:
        per_ex = big.loc[~mask].groupby("exchange").size().sort_values(ascending=False)
        print(f"[AGG] filtered outliers: {dropped} filas")
        for ex, n in per_ex.items():
            print(f"[AGG]   - {ex}: {n}")
    big = big[mask].drop(columns=["med_close"]) if dropped>0 else big.drop(columns=["med_close"]) 

    big["high"] = big[["high","open","close"]].max(axis=1)
    big["low"]  = big[["low","open","close"]].min(axis=1)

    big["w_usd"] = big["close"] * big["volume"]
    big.loc[big["exchange"].str.contains("Hyperliquid", case=False, na=False), "w_usd"] *= HYPERLIQUID_WEIGHT_PENALTY

    grp = big.groupby("date")
    open_vw  = grp.apply(lambda g: np.average(g["open"],  weights=g["w_usd"]))
    close_vw = grp.apply(lambda g: np.average(g["close"], weights=g["w_usd"]))
    high_max = grp["high"].max()
    low_min  = grp["low"].min()
    vol_usd_sum = grp["w_usd"].sum()

    out = pd.DataFrame({"open": open_vw, "close": close_vw, "high": high_max, "low": low_min, "volume_usd": vol_usd_sum})
    out.reset_index(inplace=True)
    out["timestamp"] = (out["date"].dt.tz_convert("UTC") if pd.api.types.is_datetime64tz_dtype(out["date"]) else pd.to_datetime(out["date"], utc=True))
    out["symbol"] = "BTC/USD-AGG"
    out["exchange"] = "AGGREGATED"
    print(f"[AGG] aggregated days: {len(out)}; rango: {out['timestamp'].min().date()} → {out['timestamp'].max().date()} (vol_usd sum ok)")
    return out[["timestamp","open","high","low","close","volume_usd","symbol","exchange"]]


def plot_aggregated_candles(df_agg: pd.DataFrame, title: str = "BTC Aggregated (Vol-Weighted)") -> go.Figure:
    fig = go.Figure(data=[go.Candlestick(x=df_agg["timestamp"], open=df_agg["open"], high=df_agg["high"], low=df_agg["low"], close=df_agg["close"], name="AGG")])
    fig.update_layout(title=title, xaxis_rangeslider_visible=False, height=600, hovermode='x unified')
    return fig


def plot_premiums(coinbase_prem: pd.DataFrame, kimchi_prem: pd.DataFrame) -> go.Figure:
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.07, subplot_titles=("Coinbase Premium (%)", "Kimchi Premium (%)"))
    if coinbase_prem is not None and not coinbase_prem.empty:
        fig.add_trace(go.Scatter(x=coinbase_prem["date"], y=coinbase_prem["coinbase_premium_pct"], name="CB Prem %"), row=1, col=1)
    if kimchi_prem is not None and not kimchi_prem.empty:
        fig.add_trace(go.Scatter(x=kimchi_prem["date"], y=kimchi_prem["kimchi_pct"], name="Kimchi %"), row=2, col=1)
    fig.update_layout(height=600, hovermode='x unified')
    return fig


# ============================= Visualizaciones mejoradas ==============================

def plot_aggregated_candles_with_volume(df_agg: pd.DataFrame, title: str = "BTC Aggregated (Vol-Weighted)") -> go.Figure:
    if df_agg.empty:
        print("[PLOT] No aggregated data to plot")
        return go.Figure()
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        row_heights=[0.7, 0.3],
        subplot_titles=('BTC Aggregated Price (Volume-Weighted)', 'Aggregated Volume (USD)')
    )
    fig.add_trace(
        go.Candlestick(
            x=df_agg["timestamp"],
            open=df_agg["open"],
            high=df_agg["high"],
            low=df_agg["low"],
            close=df_agg["close"],
            name="Price",
            increasing_line_color='#26a69a',
            decreasing_line_color='#ef5350',
            showlegend=False
        ),
        row=1, col=1
    )
    colors = ['#ef5350' if close < open else '#26a69a' for close, open in zip(df_agg['close'], df_agg['open'])]
    volume_col = 'volume_usd' if 'volume_usd' in df_agg.columns else 'volume'
    fig.add_trace(
        go.Bar(
            x=df_agg["timestamp"],
            y=df_agg[volume_col],
            name='Volume',
            marker_color=colors,
            opacity=0.5,
            showlegend=False,
            hovertemplate='Volume: $%{y:,.0f}<extra></extra>'
        ),
        row=2, col=1
    )
    fig.update_layout(
        title={'text': title,'x': 0.5,'xanchor': 'center','font': {'size': 20}},
        xaxis_rangeslider_visible=False,
        height=700,
        hovermode='x unified',
        plot_bgcolor='#ffffff',
        paper_bgcolor='#f8f9fa'
    )
    fig.update_xaxes(title_text="Date", gridcolor='#e5e7eb', row=2, col=1)
    fig.update_yaxes(title_text="Price (USD)", tickformat='$,.0f', gridcolor='#e5e7eb', row=1, col=1)
    fig.update_yaxes(title_text="Volume (USD)", tickformat='$,.0f', gridcolor='#e5e7eb', row=2, col=1)
    if not df_agg.empty:
        latest_price = df_agg['close'].iloc[-1]
        total_volume = df_agg[volume_col].sum()
        avg_volume = df_agg[volume_col].mean()
        annotation_text = (f"Latest: ${latest_price:,.2f} | Avg Daily Vol: ${avg_volume:,.0f} | Total Vol: ${total_volume/1e9:.2f}B")
        fig.add_annotation(text=annotation_text, xref="paper", yref="paper", x=0.5, y=1.05, showarrow=False, font=dict(size=12, color='#666'), xanchor='center')
    return fig


def create_individual_pair_charts(data: Dict[str, pd.DataFrame], charts_dir: str = "exchange_charts") -> Dict[str, go.Figure]:
    os.makedirs(charts_dir, exist_ok=True)
    charts = {}
    print("\n=== Creating Individual Pair Charts ===")
    for key, df in data.items():
        if df is None or df.empty:
            continue
        exchange, symbol = key.split(":", 1)
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, row_heights=[0.7, 0.3], subplot_titles=(f'{exchange} {symbol} Price', 'Volume'))
        fig.add_trace(
            go.Candlestick(
                x=df['timestamp'], open=df['open'], high=df['high'], low=df['low'], close=df['close'],
                name='Price', increasing_line_color='#26a69a', decreasing_line_color='#ef5350', showlegend=False
            ),
            row=1, col=1
        )
        colors = ['#ef5350' if close < open else '#26a69a' for close, open in zip(df['close'], df['open'])]
        fig.add_trace(go.Bar(x=df['timestamp'], y=df['volume'], name='Volume', marker_color=colors, opacity=0.5, showlegend=False), row=2, col=1)
        latest_price = df['close'].iloc[-1]
        price_change = ((df['close'].iloc[-1] / df['close'].iloc[0]) - 1) * 100 if len(df) > 1 else 0
        avg_volume = df['volume'].mean()
        max_price = df['high'].max()
        min_price = df['low'].min()
        title_text = f'{exchange} - {symbol} (1D)'
        fig.update_layout(title={'text': title_text,'x': 0.5,'xanchor': 'center','font': {'size': 18}},
                          xaxis_rangeslider_visible=False, height=600, hovermode='x unified', plot_bgcolor='#ffffff', paper_bgcolor='#f8f9fa')
        fig.update_xaxes(title_text="Date", gridcolor='#e5e7eb', row=2, col=1)
        currency = "KRW" if "KRW" in symbol else "USD"
        currency_symbol = "₩" if currency == "KRW" else "$"
        fig.update_yaxes(title_text=f"Price ({currency})", tickformat=f'{currency_symbol},.0f', gridcolor='#e5e7eb', row=1, col=1)
        fig.update_yaxes(title_text="Volume (BTC)" if currency != "KRW" else "Volume", tickformat=',.2f', gridcolor='#e5e7eb', row=2, col=1)
        stats_text = (f"Latest: {currency_symbol}{latest_price:,.2f} | Change: {price_change:+.1f}% | "
                      f"Range: {currency_symbol}{min_price:,.0f}-{currency_symbol}{max_price:,.0f} | Avg Vol: {avg_volume:.2f}")
        fig.add_annotation(text=stats_text, xref="paper", yref="paper", x=0.5, y=1.08, showarrow=False, font=dict(size=11, color='#666'), xanchor='center')
        data_info = f"Data points: {len(df)} | Period: {df['timestamp'].min().date()} to {df['timestamp'].max().date()}"
        fig.add_annotation(text=data_info, xref="paper", yref="paper", x=0.5, y=-0.15, showarrow=False, font=dict(size=10, color='#999'), xanchor='center')
        charts[key] = fig
        filename = f"{exchange.lower()}_{symbol.replace('/', '_').lower()}_chart.html"
        filepath = os.path.join(charts_dir, filename)
        fig.write_html(filepath)
        print(f"  ✓ Created chart for {exchange} {symbol} → {filename}")
    print(f"Total individual charts created: {len(charts)}")
    return charts


def create_comparison_grid(data: Dict[str, pd.DataFrame], pairs_to_compare: list = None) -> go.Figure:
    if pairs_to_compare is None:
        pairs_to_compare = ["BTC/USDT", "BTC/USDC", "BTC/USD", "BTC/KRW"]
    fig_dict = {}
    for target_symbol in pairs_to_compare:
        exchanges_with_pair = []
        for key, df in data.items():
            if df is None or df.empty:
                continue
            exchange, symbol = key.split(":", 1)
            if symbol.replace("-", "/").upper() == target_symbol.upper():
                exchanges_with_pair.append((exchange, df))
        if len(exchanges_with_pair) > 1:
            n_ex = len(exchanges_with_pair)
            fig = make_subplots(rows=n_ex, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                                subplot_titles=[f"{ex} - {target_symbol}" for ex, _ in exchanges_with_pair],
                                row_heights=[1/n_ex] * n_ex)
            for idx, (exchange, df) in enumerate(exchanges_with_pair, 1):
                fig.add_trace(
                    go.Candlestick(x=df['timestamp'], open=df['open'], high=df['high'], low=df['low'], close=df['close'],
                                   name=exchange, increasing_line_color='#26a69a', decreasing_line_color='#ef5350', showlegend=False),
                    row=idx, col=1
                )
            fig.update_layout(title=f'Exchange Comparison: {target_symbol}', xaxis_rangeslider_visible=False,
                              height=200 * n_ex, hovermode='x unified')
            fig.update_xaxes(title_text="Date", row=n_ex, col=1)
            for idx in range(1, n_ex + 1):
                currency = "KRW" if "KRW" in target_symbol else "USD"
                fig.update_yaxes(title_text=f"Price ({currency})", row=idx, col=1)
            fig_dict[target_symbol] = fig
    return fig_dict


def create_all_visualizations(data: Dict[str, pd.DataFrame], df_agg: pd.DataFrame, 
                              coinbase_prem: pd.DataFrame, kimchi_prem: pd.DataFrame,
                              charts_dir: str = "exchange_charts") -> Dict:
    os.makedirs(charts_dir, exist_ok=True)
    results = {'individual_charts': {}, 'aggregated_chart': None, 'premium_charts': None, 'comparison_grids': {}}
    print("\n=== Creating Aggregated Chart with Volume ===")
    fig_agg = plot_aggregated_candles_with_volume(df_agg)
    agg_path = os.path.join(charts_dir, "btc_aggregated_with_volume.html")
    fig_agg.write_html(agg_path)
    results['aggregated_chart'] = agg_path
    print(f"  ✓ Saved aggregated chart → {agg_path}")
    individual_charts = create_individual_pair_charts(data, charts_dir)
    results['individual_charts'] = individual_charts
    print("\n=== Creating Comparison Grids ===")
    comparison_grids = create_comparison_grid(data)
    for symbol, fig in comparison_grids.items():
        filename = f"comparison_{symbol.replace('/', '_').lower()}.html"
        filepath = os.path.join(charts_dir, filename)
        fig.write_html(filepath)
        results['comparison_grids'][symbol] = filepath
        print(f"  ✓ Created comparison for {symbol} → {filename}")
    fig_premiums = plot_premiums(coinbase_prem, kimchi_prem)
    premium_path = os.path.join(charts_dir, "premiums_cb_kimchi.html")
    fig_premiums.write_html(premium_path)
    results['premium_charts'] = premium_path
    print(f"\n=== Visualization Summary ===")
    print(f"Total charts created: {len(individual_charts) + len(comparison_grids) + 2}")
    print(f"Location: {os.path.abspath(charts_dir)}")
    return results


In [8]:
# ======================================================================================
# Tablas con estilo (rich + HTML)
# ======================================================================================

def _style_html_table(df: pd.DataFrame, title: str) -> str:
    if df.empty:
        return f"<h3>{title}</h3><p>No data available</p>"
    def format_value(val, col):
        if pd.isna(val):
            return ""
        if col in ["share_change_pp", "slope_pp_per_30d"]:
            return f"{val:+.2f}"
        elif col == "avg_weight":
            return f"{val*100:.2f}%"
        else:
            return str(val)
    def get_color_for_change(val):
        if pd.isna(val):
            return ""
        if val > 0:
            intensity = min(abs(val) / 10, 1) * 0.3
            return f"background-color: rgba(34, 197, 94, {intensity});"
        else:
            intensity = min(abs(val) / 10, 1) * 0.3
            return f"background-color: rgba(239, 68, 68, {intensity});"
    def get_color_for_slope(val):
        if pd.isna(val) or val <= 0:
            return ""
        intensity = min(val / 10, 1) * 0.2
        return f"background-color: rgba(59, 130, 246, {intensity});"
    def get_bar_width(val, max_val):
        if pd.isna(val) or max_val == 0:
            return 0
        return min((val / max_val) * 100, 100)
    html_rows = []
    max_weight = df["avg_weight"].max() if "avg_weight" in df.columns else 1
    for _, row in df.iterrows():
        cells = []
        for col in df.columns:
            val = row[col]
            formatted = format_value(val, col)
            style = ""
            if col == "share_change_pp":
                style = get_color_for_change(val)
            elif col == "slope_pp_per_30d":
                style = get_color_for_slope(val)
            elif col == "avg_weight" and not pd.isna(val):
                bar_width = get_bar_width(val, max_weight)
                style = f"background: linear-gradient(90deg, rgba(99, 102, 241, 0.2) {bar_width}%, transparent {bar_width}%);"
            cells.append(f'<td style="padding:6px 8px;{style}">{formatted}</td>')
        html_rows.append(f'<tr>{"".join(cells)}</tr>')
    headers = "".join([f'<th style="text-align:center;padding:6px 8px;background:#f3f4f6;">{col}</th>' for col in df.columns])
    html = f"""
    <table style="border-collapse:collapse;width:100%;margin:10px 0;border:1px solid #e5e7eb;">
        <caption style="caption-side:top;font-size:16px;font-weight:600;padding:8px 0;text-align:left;">
            {title}
        </caption>
        <thead><tr>{headers}</tr></thead>
        <tbody>{"".join(html_rows)}</tbody>
    </table>
    """
    return html

def _export_winners_losers_html(w7: pd.DataFrame, w30: pd.DataFrame, w90: pd.DataFrame, out_path: str):
    sections = []
    for label, df in [("7D", w7), ("30D", w30), ("90D", w90)]:
        if df is None or df.empty:
            continue
        winners = df.sort_values("share_change_pp", ascending=False).head(3)[["name","share_change_pp","slope_pp_per_30d","avg_weight"]]
        losers  = df.sort_values("share_change_pp", ascending=True).head(3)[["name","share_change_pp","slope_pp_per_30d","avg_weight"]]
        sections.append(f"<h2 style='font-family:system-ui;margin:16px 0 8px'>{label}</h2>")
        sections.append(_style_html_table(winners.reset_index(drop=True), f"Ganadores {label} (pp de cuota)"))
        sections.append(_style_html_table(losers.reset_index(drop=True),  f"Perdedores {label} (pp de cuota)"))
    html = f"""
    <html><head><meta charset="utf-8"><title>Ganadores/Perdedores BTC</title></head>
    <body style="font-family:system-ui; margin:24px">
      <h1 style="margin:0 0 8px">Ganadores / Perdedores por cuota</h1>
      <div style="color:#666; margin-bottom:16px">Generado automáticamente</div>
      {''.join(sections)}
    </body></html>
    """
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(html)

def _print_top(df: pd.DataFrame, label: str):
    if df is None or df.empty:
        print(f"{label}: sin datos"); return
    top_g = df.sort_values("share_change_pp", ascending=False).head(3)[["name","share_change_pp","slope_pp_per_30d","avg_weight"]]
    top_l = df.sort_values("share_change_pp", ascending=True).head(3)[["name","share_change_pp","slope_pp_per_30d","avg_weight"]]
    try:
        from rich.console import Console
        from rich.table import Table
        from rich import box
        console = Console()
        def _as_table(title: str, data: pd.DataFrame, color: str):
            t = Table(title=title, box=box.SIMPLE_HEAVY)
            for col in ["name","share_change_pp","slope_pp_per_30d","avg_weight"]:
                t.add_column(col, justify="right" if col != "name" else "left", style="bold" if col=="name" else "")
            for _, r in data.iterrows():
                t.add_row(str(r["name"]), f"{r['share_change_pp']:+.2f}", f"{r['slope_pp_per_30d']:+.2f}", f"{r['avg_weight']:.2%}", style=color)
            return t
        console.print(_as_table(f"Ganadores {label} (pp de cuota)", top_g, "green"))
        console.print(_as_table(f"Perdedores {label} (pp de cuota)", top_l, "red"))
    except Exception:
        print(f"Ganadores {label} (pp de cuota):")
        print(top_g.to_string(index=False, float_format=lambda x: f"{x:.2f}"))
        print(f"Perdedores {label} (pp de cuota):")
        print(top_l.to_string(index=False, float_format=lambda x: f"{x:.2f}"))
    return top_g, top_l


## Imports and Configuration
Importing necessary libraries and setting up global variables.

In [9]:
# ======================================================================================
# Capa incremental, pesos y utilidades de resumen
# ======================================================================================

from __future__ import annotations

import os
from typing import Optional, List, Dict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone

def ensure_dataset(exchange: str, symbol: str, fetch_fn, *fetch_args, **fetch_kwargs) -> pd.DataFrame:
    """Carga desde CSV si existe. Si ya contiene HOY → skip. Si falta hoy → baja ayer+y hoy y hace append. Con logs."""
    fname = expected_csv_name(exchange, symbol)
    fpath = os.path.join(DATA_DIR, fname)
    today = today_utc_date()
    existing = read_csv_if_exists(fpath)
    if existing is not None and not existing.empty:
        last_dates = existing["timestamp"].dt.date
        print(f"[{exchange}] {symbol} → CSV: {len(existing)} filas; última={existing['timestamp'].max().date()}")
        if (last_dates == today).any():
            print(f"[{exchange}] {symbol} → ya incluye HOY → skip")
            return existing
        print(f"[{exchange}] {symbol} → falta HOY → descargar AYER+HOY")
        start_ms = two_day_start_ms_utc() if ONLY_LAST_2_DAYS_IF_MISSING else None
        new_df = fetch_fn(*fetch_args, **{**fetch_kwargs, "start_ms": start_ms})
        if new_df is None or new_df.empty:
            print(f"[{exchange}] {symbol} → sin datos nuevos; mantengo CSV")
            return existing
        start_dt = pd.to_datetime(start_ms, unit='ms', utc=True) if start_ms else existing['timestamp'].max().floor('D')
        new_df = new_df[new_df['timestamp'] >= start_dt]
        merged = pd.concat([existing, new_df], ignore_index=True).drop_duplicates(subset=['timestamp']).sort_values('timestamp')
        added = len(merged) - len(existing)
        print(f"[{exchange}] {symbol} → añadidas {added} filas; nueva última={merged['timestamp'].max().date()}")
        save_csv(merged, fname)
        return merged
    print(f"[{exchange}] {symbol} → no hay CSV → {'histórico' if FULL_HIST_ON_FIRST_RUN else 'AYER+HOY'}")
    start_ms = None if FULL_HIST_ON_FIRST_RUN else two_day_start_ms_utc()
    df = fetch_fn(*fetch_args, **{**fetch_kwargs, "start_ms": start_ms})
    print(f"[{exchange}] {symbol} → descargadas {0 if df is None else len(df)} filas")
    save_csv(df, fname)
    return df

def compute_daily_weights(named_dfs: List[tuple]) -> pd.DataFrame:
    frames = []
    for name, df in named_dfs:
        if df is None or df.empty:
            continue
        tmp = ensure_daily(df)[["date","close","volume"]].copy()
        tmp["name"] = name
        tmp["notional_usd"] = tmp["close"] * tmp["volume"]
        if "hyperliquid" in name.lower():
            tmp["notional_usd"] *= HYPERLIQUID_WEIGHT_PENALTY
        frames.append(tmp[["date","name","notional_usd"]])
    if not frames:
        return pd.DataFrame()
    allv = pd.concat(frames, ignore_index=True)
    allv = allv.groupby(["date","name"], as_index=False)["notional_usd"].sum()
    sums = allv.groupby("date")["notional_usd"].sum().rename("tot_notional_usd")
    m = allv.merge(sums, on="date", how="left")
    m["weight"] = np.where(m["tot_notional_usd"] > 0, m["notional_usd"] / m["tot_notional_usd"], 0.0)
    return m[["date","name","notional_usd","tot_notional_usd","weight"]].sort_values(["date","weight"], ascending=[True, False])

def summarize_period_weights(weights: pd.DataFrame, days: int) -> pd.DataFrame:
    """Resumen de pesos para los últimos N días, con cambio vs. periodo previo y tendencia (pendiente mensualizada)."""
    if weights is None or weights.empty:
        return pd.DataFrame()
    last_day = weights["date"].max()
    if pd.isna(last_day):
        return pd.DataFrame()
    period_start = last_day - pd.Timedelta(days=days-1)
    cur = weights[(weights["date"] >= period_start) & (weights["date"] <= last_day)].copy()
    if cur.empty:
        return pd.DataFrame()
    prev_start = period_start - pd.Timedelta(days=days)
    prev_end = period_start - pd.Timedelta(days=1)
    prev = weights[(weights["date"] >= prev_start) & (weights["date"] <= prev_end)].copy()

    cur_agg = cur.groupby("name").agg(
        notional_usd=("notional_usd","sum"),
        avg_weight=("weight","mean"),
        days_observed=("date","count")
    ).reset_index()

    prev_w = prev.groupby("name")["weight"].mean().rename("avg_weight_prev").reset_index()
    prev_n = prev.groupby("name")["notional_usd"].sum().rename("notional_usd_prev").reset_index()

    out = cur_agg.merge(prev_w, on="name", how="left").merge(prev_n, on="name", how="left")
    out["avg_weight_prev"] = out["avg_weight_prev"].fillna(0.0)
    out["notional_usd_prev"] = out["notional_usd_prev"].fillna(0.0)
    out["share_change_pp"] = (out["avg_weight"] - out["avg_weight_prev"]) * 100.0
    out["notional_change_pct"] = np.where(out["notional_usd_prev"] > 0.0,
                                           (out["notional_usd"] - out["notional_usd_prev"]) / out["notional_usd_prev"] * 100.0,
                                           np.nan)
    slopes = []
    for name, g in cur.groupby("name"):
        if len(g) >= 2:
            x = (g["date"] - g["date"].min()).dt.days.astype(float)
            y = g["weight"].astype(float).values
            slope = float(np.polyfit(x, y, 1)[0]) * 100.0 * 30.0
        else:
            slope = np.nan
        slopes.append({"name": name, "slope_pp_per_30d": slope})
    out = out.merge(pd.DataFrame(slopes), on="name", how="left")
    return out.sort_values(["avg_weight"], ascending=[False]).reset_index(drop=True)


In [10]:
# ======================================================================================
# Métodos de chequeo/diagnóstico de pares por exchange (como OKX), en celda independiente
# ======================================================================================

# ---------- OKX ----------
def check_okx_pair_availability(symbol: str = "BTC/USDC") -> Dict:
    instruments_url = "https://www.okx.com/api/v5/public/instruments"
    okx_symbol = symbol.replace("/", "-")
    params = {"instType": "SPOT", "instId": okx_symbol}
    try:
        r = requests.get(instruments_url, params=params, headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            js = r.json()
            if js.get("code") == "0":
                data = js.get("data", [])
                if data:
                    inst = data[0]
                    return {
                        "available": True,
                        "symbol": inst.get("instId"),
                        "baseCcy": inst.get("baseCcy"),
                        "quoteCcy": inst.get("quoteCcy"),
                        "state": inst.get("state"),
                        "minSize": inst.get("minSz"),
                        "listTime": inst.get("listTime"),
                        "alias": inst.get("alias")
                    }
    except Exception as e:
        print(f"[OKX] Error checking pair availability: {e}")
    return {"available": False, "symbol": okx_symbol}

def diagnose_okx_pair(symbol: str = "BTC/USDC"):
    print(f"\n[OKX DIAGNOSTIC] Checking {symbol}...")
    info = check_okx_pair_availability(symbol)
    if not info.get("available"):
        print(f"[OKX DIAGNOSTIC] ❌ Pair {symbol} NOT FOUND on OKX Spot")
        print(f"[OKX DIAGNOSTIC] Suggestion: Use BTC/USDT instead, which has better liquidity")
        return
    print(f"[OKX DIAGNOSTIC] ✅ Pair found: {info.get('symbol')}")
    print(f"[OKX DIAGNOSTIC] State: {info.get('state')}")
    if info.get('listTime'):
        list_date = pd.to_datetime(int(info['listTime']), unit='ms')
        days_since_listing = (datetime.now() - list_date).days
        print(f"[OKX DIAGNOSTIC] Listed: {list_date.date()} ({days_since_listing} days ago)")
        if days_since_listing < 30:
            print(f"[OKX DIAGNOSTIC] ⚠️ This is a NEWLY LISTED pair (< 30 days)")
    ticker_url = "https://www.okx.com/api/v5/market/ticker"
    params = {"instId": info.get('symbol')}
    try:
        r = requests.get(ticker_url, params=params, headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            js = r.json()
            if js.get("code") == "0" and js.get("data"):
                ticker = js["data"][0]
                vol24h = float(ticker.get("vol24h", 0))
                volCcy24h = float(ticker.get("volCcy24h", 0))
                print(f"[OKX DIAGNOSTIC] 24h Volume: {vol24h:,.2f} BTC (${volCcy24h:,.0f})")
                if volCcy24h < 100000:
                    print(f"[OKX DIAGNOSTIC] ⚠️ LOW LIQUIDITY: 24h volume < $100k")
    except Exception as e:
        print(f"[OKX DIAGNOSTIC] Could not fetch volume data: {e}")
    print("[OKX DIAGNOSTIC] Complete\n")

# ---------- Binance ----------
def check_binance_pair_availability(symbol: str = "BTC/USDT") -> Dict:
    url = "https://api.binance.com/api/v3/exchangeInfo"
    sym = symbol.replace("/", "")
    try:
        r = requests.get(url, params={"symbol": sym}, headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            js = r.json()
            if js.get("symbols"):
                s = js["symbols"][0]
                return {"available": True, "symbol": s.get("symbol"), "base": s.get("baseAsset"),
                        "quote": s.get("quoteAsset"), "status": s.get("status")}
    except Exception as e:
        print(f"[Binance] Availability error: {e}")
    return {"available": False, "symbol": sym}

def diagnose_binance_pair(symbol: str = "BTC/USDT"):
    print(f"\n[Binance DIAGNOSTIC] Checking {symbol}...")
    info = check_binance_pair_availability(symbol)
    if not info.get("available"):
        print(f"[Binance DIAGNOSTIC] ❌ Pair not found"); return
    print(f"[Binance DIAGNOSTIC] ✅ {info['symbol']} | status={info.get('status')} base={info.get('base')} quote={info.get('quote')}")
    # 24h ticker
    try:
        r = requests.get("https://api.binance.com/api/v3/ticker/24hr",
                         params={"symbol": info['symbol']}, headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            t = r.json()
            print(f"[Binance DIAGNOSTIC] 24h vol: {float(t.get('volume',0)):.4f} {info.get('base')} | quoteVol: {float(t.get('quoteVolume',0)):.0f}")
    except Exception as e:
        print(f"[Binance DIAGNOSTIC] Ticker error: {e}")

# ---------- Bybit ----------
def check_bybit_pair_availability(symbol: str = "BTC/USDT") -> Dict:
    url = "https://api.bybit.com/v5/market/instruments-info"
    sym = symbol.replace("/", "")
    try:
        r = requests.get(url, params={"category":"spot","symbol": sym}, headers=USER_AGENT, timeout=30)
        js = r.json()
        if js.get("retCode") == 0 and js.get("result", {}).get("list"):
            s = js["result"]["list"][0]
            return {"available": True, "symbol": s.get("symbol"), "status": s.get("status")}
    except Exception as e:
        print(f"[Bybit] Availability error: {e}")
    return {"available": False, "symbol": sym}

def diagnose_bybit_pair(symbol: str = "BTC/USDT"):
    print(f"\n[Bybit DIAGNOSTIC] Checking {symbol}...")
    info = check_bybit_pair_availability(symbol)
    if not info.get("available"):
        print("[Bybit DIAGNOSTIC] ❌ Pair not found"); return
    print(f"[Bybit DIAGNOSTIC] ✅ {info['symbol']} | status={info.get('status')}")
    try:
        r = requests.get("https://api.bybit.com/v5/market/tickers",
                         params={"category":"spot","symbol": info['symbol']}, headers=USER_AGENT, timeout=30)
        js = r.json()
        if js.get("retCode") == 0 and js.get("result",{}).get("list"):
            t = js["result"]["list"][0]
            print(f"[Bybit DIAGNOSTIC] 24h vol: {float(t.get('volume24h',0)):.4f}")
    except Exception as e:
        print(f"[Bybit DIAGNOSTIC] Ticker error: {e}")

# ---------- Coinbase ----------
def check_coinbase_product_availability(symbol: str = "BTC/USD") -> Dict:
    prod = CoinbaseFetcher._pick_product(symbol)
    url = f"https://api.exchange.coinbase.com/products/{prod}"
    try:
        r = requests.get(url, headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            js = r.json()
            return {"available": True, "id": js.get("id"), "base": js.get("base_currency"), "quote": js.get("quote_currency"),
                    "status": js.get("status")}
    except Exception as e:
        print(f"[Coinbase] Availability error: {e}")
    return {"available": False, "id": prod}

def diagnose_coinbase_product(symbol: str = "BTC/USD"):
    print(f"\n[Coinbase DIAGNOSTIC] Checking {symbol}...")
    info = check_coinbase_product_availability(symbol)
    if not info.get("available"):
        print("[Coinbase DIAGNOSTIC] ❌ Product not found"); return
    print(f"[Coinbase DIAGNOSTIC] ✅ {info['id']} | status={info.get('status')} base={info.get('base')} quote={info.get('quote')}")
    try:
        r = requests.get(f"https://api.exchange.coinbase.com/products/{info['id']}/ticker", headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            t = r.json()
            print(f"[Coinbase DIAGNOSTIC] 24h vol: {float(t.get('volume',0)):.4f}")
    except Exception as e:
        print(f"[Coinbase DIAGNOSTIC] Ticker error: {e}")

# ---------- Crypto.com ----------
def check_crypto_com_pair_availability(symbol: str = "BTC/USDT") -> Dict:
    url = "https://api.crypto.com/exchange/v1/public/get-instruments"
    inst = symbol.replace("/", "_")
    try:
        r = requests.get(url, headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            js = r.json()
            data = js.get("result", {}).get("instruments", [])
            ok = any(x.get("instrument_name") == inst for x in data)
            return {"available": ok, "instrument_name": inst}
    except Exception as e:
        print(f"[Crypto.com] Availability error: {e}")
    return {"available": False, "instrument_name": inst}

def diagnose_crypto_com_pair(symbol: str = "BTC/USDT"):
    print(f"\n[Crypto.com DIAGNOSTIC] Checking {symbol}...")
    info = check_crypto_com_pair_availability(symbol)
    if not info.get("available"):
        print("[Crypto.com DIAGNOSTIC] ❌ Pair not found"); return
    print(f"[Crypto.com DIAGNOSTIC] ✅ {info['instrument_name']}")
    try:
        r = requests.get("https://api.crypto.com/exchange/v1/public/get-ticker",
                         params={"instrument_name": info['instrument_name']}, headers=USER_AGENT, timeout=30)
        js = r.json()
        if js.get("code") == 0 and js.get("result",{}).get("data"):
            t = js["result"]["data"][0]
            print(f"[Crypto.com DIAGNOSTIC] 24h vol: {float(t.get('v',0)):.4f}")
    except Exception as e:
        print(f"[Crypto.com DIAGNOSTIC] Ticker error: {e}")

# ---------- Hyperliquid ----------
def check_hyperliquid_pair_availability(symbol: str = "BTC/USDC") -> Dict:
    coin = symbol.split('/')[0].upper()
    payload = {"type": "candleSnapshot", "req": {"coin": coin, "interval": "1d",
                                                 "startTime": to_ms(datetime.now(timezone.utc)-timedelta(days=2)),
                                                 "endTime": to_ms(datetime.now(timezone.utc))}}
    try:
        js = HyperliquidFetcher._post(payload)
        ok = isinstance(js, list) and len(js) > 0
        return {"available": ok, "coin": coin}
    except Exception as e:
        print(f"[Hyperliquid] Availability error: {e}")
    return {"available": False, "coin": coin}

def diagnose_hyperliquid_pair(symbol: str = "BTC/USDC"):
    print(f"\n[Hyperliquid DIAGNOSTIC] Checking {symbol}...")
    info = check_hyperliquid_pair_availability(symbol)
    if not info.get("available"):
        print("[Hyperliquid DIAGNOSTIC] ❌ Pair not available"); return
    print(f"[Hyperliquid DIAGNOSTIC] ✅ {info['coin']}")
    # No simple public 24h ticker USD endpoint; candle snapshot acts as basic check.

# ---------- Upbit ----------
def check_upbit_pair_availability(symbol: str = "BTC/KRW") -> Dict:
    market = "KRW-BTC" if symbol.upper().endswith("KRW") else ("USDT-BTC" if symbol.upper().endswith("USDT") else "KRW-BTC")
    try:
        r = requests.get("https://api.upbit.com/v1/market/all", headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            arr = r.json()
            ok = any(x.get("market") == market for x in arr)
            return {"available": ok, "market": market}
    except Exception as e:
        print(f"[Upbit] Availability error: {e}")
    return {"available": False, "market": market}

def diagnose_upbit_pair(symbol: str = "BTC/KRW"):
    print(f"\n[Upbit DIAGNOSTIC] Checking {symbol}...")
    info = check_upbit_pair_availability(symbol)
    if not info.get("available"):
        print("[Upbit DIAGNOSTIC] ❌ Market not found"); return
    print(f"[Upbit DIAGNOSTIC] ✅ {info['market']}")
    try:
        r = requests.get("https://api.upbit.com/v1/ticker", params={"markets": info['market']}, headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            t = r.json()[0]
            print(f"[Upbit DIAGNOSTIC] 24h vol: {float(t.get('acc_trade_volume_24h',0)):.4f}")
    except Exception as e:
        print(f"[Upbit DIAGNOSTIC] Ticker error: {e}")

# ---------- Bitget ----------
def check_bitget_pair_availability(symbol: str = "BTC/USDT") -> Dict:
    sym = symbol.replace("/", "")
    try:
        r = requests.get("https://api.bitget.com/api/v2/spot/public/symbols", headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            js = r.json()
            data = js.get("data", [])
            ok = any(x.get("symbol") == sym for x in data)
            return {"available": ok, "symbol": sym}
    except Exception as e:
        print(f"[Bitget] Availability error: {e}")
    return {"available": False, "symbol": sym}

def diagnose_bitget_pair(symbol: str = "BTC/USDT"):
    print(f"\n[Bitget DIAGNOSTIC] Checking {symbol}...")
    info = check_bitget_pair_availability(symbol)
    if not info.get("available"):
        print("[Bitget DIAGNOSTIC] ❌ Pair not found"); return
    print(f"[Bitget DIAGNOSTIC] ✅ {info['symbol']}")
    try:
        r = requests.get("https://api.bitget.com/api/v2/spot/market/tickers", params={"symbol": info['symbol']}, headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            js = r.json()
            data = js.get("data", [])
            if data:
                t = data[0]
                print(f"[Bitget DIAGNOSTIC] 24h vol: {float(t.get('baseVol',0)):.4f}")
    except Exception as e:
        print(f"[Bitget DIAGNOSTIC] Ticker error: {e}")

# ---------- MEXC ----------
def check_mexc_pair_availability(symbol: str = "BTC/USDT") -> Dict:
    sym = symbol.replace("/", "")
    try:
        r = requests.get("https://api.mexc.com/api/v3/exchangeInfo", params={"symbol": sym}, headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            js = r.json()
            s = js.get("symbols", [])
            if s:
                rec = s[0]
                return {"available": True, "symbol": rec.get("symbol"), "status": rec.get("status")}
    except Exception as e:
        print(f"[MEXC] Availability error: {e}")
    return {"available": False, "symbol": sym}

def diagnose_mexc_pair(symbol: str = "BTC/USDT"):
    print(f"\n[MEXC DIAGNOSTIC] Checking {symbol}...")
    info = check_mexc_pair_availability(symbol)
    if not info.get("available"):
        print("[MEXC DIAGNOSTIC] ❌ Pair not found"); return
    print(f"[MEXC DIAGNOSTIC] ✅ {info['symbol']} | status={info.get('status')}")
    try:
        r = requests.get("https://api.mexc.com/api/v3/ticker/24hr", params={"symbol": info['symbol']}, headers=USER_AGENT, timeout=30)
        if r.status_code == 200:
            t = r.json()
            print(f"[MEXC DIAGNOSTIC] 24h vol: {float(t.get('volume',0)):.4f}")
    except Exception as e:
        print(f"[MEXC DIAGNOSTIC] Ticker error: {e}")


## Main Execution
Entry point to run the data collection and analysis pipeline.

In [11]:
# ================================ MAIN =====================================

def main():
    diagnose_okx_pair("BTC/USDC")
    start_hist_ms = to_ms(datetime(2019,9,1,tzinfo=timezone.utc))
    print("=== Downloading/Updating 1D candles ===")
    tasks = [
        ("Binance", "BTC/USDT", BinanceFetcher.fetch, {"interval":"1d", "start_ms": start_hist_ms}),
        ("Binance", "BTC/USDC", BinanceFetcher.fetch, {"interval":"1d", "start_ms": start_hist_ms}),
        ("Bybit",   "BTC/USDT", BybitFetcher.fetch,   {"interval":"D",   "start_ms": start_hist_ms}),
        ("Bybit",   "BTC/USDC", BybitFetcher.fetch,   {"interval":"D",   "start_ms": start_hist_ms}),
        ("OKX", "BTC/USDT", OKXFetcher.fetch, {"interval":"1D", "start_ms": start_hist_ms}),
        ("OKX", "BTC/USDC", OKXFetcher.fetch, {"interval":"1D", "start_ms": start_hist_ms}),
        ("Coinbase","BTC/USD",  CoinbaseFetcher.fetch,{"granularity_sec":86400, "start_ms": start_hist_ms, "include_partial_today": True}),
        ("Crypto.com","BTC/USDT", CryptoComFetcher.fetch, {"interval":"1D", "start_ms": start_hist_ms}),
        ("Crypto.com","BTC/USDC", CryptoComFetcher.fetch, {"interval":"1D", "start_ms": start_hist_ms}),
        ("Hyperliquid","BTC/USDC", HyperliquidFetcher.fetch, {"interval":"1d", "start_ms": start_hist_ms}),
        #("Hyperliquid","BTC/USDT", HyperliquidFetcher.fetch, {"interval":"1d", "start_ms": start_hist_ms}),
        ("Upbit",   "BTC/KRW",  UpbitFetcher.fetch_days, {"start_ms": start_hist_ms}),
        # Nuevos exchanges
        ("Bitget",  "BTC/USDT", BitgetFetcher.fetch, {"interval": "1D", "start_ms": start_hist_ms}),
        ("Bitget",  "BTC/USDC", BitgetFetcher.fetch, {"interval": "1D", "start_ms": start_hist_ms}),
        ("MEXC",    "BTC/USDT", MEXCFetcher.fetch,   {"interval": "1d", "start_ms": start_hist_ms}),
        ("MEXC",    "BTC/USDC", MEXCFetcher.fetch,   {"interval": "1d", "start_ms": start_hist_ms}),
    ]

    data: Dict[str, pd.DataFrame] = {}
    named_usd_datasets: List[tuple] = []

    for exch, sym, fn, kwargs in tasks:
        df = ensure_dataset(exch, sym, fn, sym, **kwargs)
        key = f"{exch}:{sym}"
        data[key] = df

    # Referencia USD (excluye Coinbase para premium CB). USD-like = USD/USDT/USDC, excluye Upbit.
    def is_usd_like(exch: str, sym: str) -> bool:
        return (sym.endswith("/USD") or sym.endswith("/USDT") or sym.endswith("/USDC")) and exch != "Upbit"

    # Canonicalización para evitar duplicados de la misma fuente
    # - Hyperliquid publica un único mercado (USD-margined), así que unificamos USDT/USDC → "BTC/USD".
    # - Para cualquier exchange: si por error llegaran dos datasets con el mismo nombre canónico,
    #   se fusionan por timestamp para evitar duplicados en los pesos.
    def canonicalize_pair(exchange: str, symbol: str) -> str:
        s = symbol.upper().replace("-", "/").strip()
        if exchange == "Hyperliquid":
            return "BTC/USD"
        return s

    named_usd_map: Dict[str, pd.DataFrame] = {}
    for key, df in data.items():
        if df is None or df.empty:
            continue
        exch, sym = key.split(":", 1)
        if (sym.endswith("/USD") or sym.endswith("/USDT") or sym.endswith("/USDC")) and exch != "Upbit":
            cname = f"{exch} {canonicalize_pair(exch, sym)}"  # p.ej. "MEXC BTC/USDT"
            if cname in named_usd_map:
                merged = pd.concat([named_usd_map[cname], df], ignore_index=True)
                merged = merged.drop_duplicates(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)
                named_usd_map[cname] = merged
            else:
                named_usd_map[cname] = df

    named_usd_datasets = list(named_usd_map.items())


    print("=== Descargando USDKRW ===")
    all_dfs_for_range = [df for df in data.values() if df is not None and not df.empty]
    min_date = min(d["timestamp"].min().date() for d in all_dfs_for_range)
    max_date = max(d["timestamp"].max().date() for d in all_dfs_for_range)
    fx_df = fetch_usdkrw_timeseries(min_date.isoformat(), max_date.isoformat())

    print("=== Calculando Coinbase Premium ===")
    coin_df = data.get("Coinbase:BTC/USD")
    refs_ex_coin = [df for name, df in named_usd_datasets if not name.startswith("Coinbase ")]
    cb_prem = compute_coinbase_premium(coin_df, refs_ex_coin) if coin_df is not None and not coin_df.empty and refs_ex_coin else pd.DataFrame()
    save_csv(cb_prem, "coinbase_premium_daily.csv")

    print("=== Calculando Kimchi Premium ===")
    if refs_ex_coin:
        # Construir referencia USD con pesos en USD y penalización a Hyperliquid
        parts = []
        for df_part in refs_ex_coin:
            dd = ensure_daily(df_part)[["date","close","volume","exchange"]].copy()
            dd["w_usd"] = dd["close"] * dd["volume"]
            dd.loc[dd["exchange"].str.contains("Hyperliquid", case=False, na=False), "w_usd"] *= HYPERLIQUID_WEIGHT_PENALTY
            parts.append(dd[["date","close","w_usd"]])
        ref_concat = pd.concat(parts, ignore_index=True)
        ref_usd = ref_concat.groupby("date").apply(lambda g: pd.Series({
            "close": (g["close"]*g["w_usd"]).sum()/max(g["w_usd"].sum(),1e-12),
            "volume_usd": g["w_usd"].sum()
        })).reset_index()
        ref_usd["timestamp"] = (ref_usd["date"].dt.tz_convert("UTC") if pd.api.types.is_datetime64tz_dtype(ref_usd["date"]) else pd.to_datetime(ref_usd["date"], utc=True))
        ref_usd_df = pd.DataFrame({
            "timestamp": ref_usd["timestamp"],
            "open": ref_usd["close"],
            "high": ref_usd["close"],
            "low": ref_usd["close"],
            "close": ref_usd["close"],
            "volume": ref_usd["volume_usd"],
            "symbol": "REF/USD", "exchange":"REF"
        })
    else:
        ref_usd_df = pd.DataFrame()

    upbit_df = data.get("Upbit:BTC/KRW", pd.DataFrame())
    kimchi = compute_kimchi_premium(upbit_df, ref_usd_df, fx_df) if not upbit_df.empty and not ref_usd_df.empty and not fx_df.empty else pd.DataFrame()
    save_csv(kimchi, "kimchi_premium_daily.csv")

    print("=== Construyendo vela agregada (USD) ===")
    usd_dfs = [df for _, df in named_usd_datasets]
    agg = aggregate_usd_candles(usd_dfs)
    save_csv(agg, "aggregated_btc_usd_1d.csv")

    print("=== Pesos por exchange (diario) ===")
    weights_input = [(name, df) for name, df in named_usd_datasets]
    weights = compute_daily_weights(weights_input)
    save_csv(weights, "aggregate_weights_daily.csv")
    if not weights.empty:
        last_day = weights["date"].max()
        w_last = weights[weights["date"] == last_day][["name","weight"]].sort_values("weight", ascending=False)
        print("Pesos última fecha:")
        print(w_last.to_string(index=False, float_format=lambda x: f"{x*100:.2f}%"))

    # === Pesos 7D/30D/90D y análisis de tendencia ===
    print("=== Pesos 7D/30D/90D y tendencias ===")
    w7  = summarize_period_weights(weights, 7)
    w30 = summarize_period_weights(weights, 30)
    w90 = summarize_period_weights(weights, 90)
    save_csv(w7,  "aggregate_weights_last7d.csv")
    save_csv(w30, "aggregate_weights_last30d.csv")
    save_csv(w90, "aggregate_weights_last90d.csv")

    _print_top(w7,  "7D")
    _print_top(w30, "30D")
    _print_top(w90, "90D")

    # Exportar HTML con estilo de ganadores/perdedores
    winners_html = os.path.join(CHARTS_DIR, "winners_losers.html")
    _export_winners_losers_html(w7, w30, w90, winners_html)
    print(" - Tablas (HTML):", winners_html)

    # === Visualizaciones mejoradas ===
    print("=== Creando todas las visualizaciones ===")
    
    # Usa la nueva función integrada
    viz_results = create_all_visualizations(
        data=data,
        df_agg=agg,
        coinbase_prem=cb_prem,
        kimchi_prem=kimchi,
        charts_dir=CHARTS_DIR
    )
    
    print("\n=== Resumen de Visualizaciones ===")
    print(f"✓ Gráfico agregado con volumen: {viz_results['aggregated_chart']}")
    print(f"✓ Gráficos individuales creados: {len(viz_results['individual_charts'])}")
    print(f"✓ Grids de comparación: {len(viz_results['comparison_grids'])}")
    print(f"✓ Ubicación: {os.path.abspath(CHARTS_DIR)}")


if __name__ == "__main__":
    main()



[OKX DIAGNOSTIC] Checking BTC/USDC...
[OKX DIAGNOSTIC] ✅ Pair found: BTC-USDC
[OKX DIAGNOSTIC] State: live
[OKX DIAGNOSTIC] Listed: 2025-08-21 (78 days ago)
[OKX DIAGNOSTIC] 24h Volume: 6.68 BTC ($680,873)
[OKX DIAGNOSTIC] Complete

=== Descargando/Actualizando velas 1D ===
[Binance] BTC/USDT → no hay CSV → histórico
[Binance] BTC/USDT → descargadas 2260 filas
[Binance] BTC/USDC → no hay CSV → histórico
[Binance] BTC/USDC → descargadas 2097 filas
[Bybit] BTC/USDT → no hay CSV → histórico
[Bybit] BTC/USDT → descargadas 1587 filas
[Bybit] BTC/USDC → no hay CSV → histórico
[Bybit] BTC/USDC → descargadas 1430 filas
[OKX] BTC/USDT → no hay CSV → histórico
[OKX] Fetching BTC/USDT interval=1D
[OKX] Step 1: Fetching recent data for BTC/USDT...
[OKX] Got 100 recent candles
[OKX] Recent data range: 2025-07-30 → 2025-11-06
[OKX] Step 2: Fetching historical data for BTC/USDT...
[OKX] Historical: 5 pages, 601 total candles, oldest so far: 2024-03-16
[OKX] Historical: 10 pages, 1101 total candles, 

C:\Users\Pedro\AppData\Local\Temp\ipykernel_378520\3912009310.py:66: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = ref.groupby("date").apply(lambda g: pd.Series({


=== Calculando Kimchi Premium ===


C:\Users\Pedro\AppData\Local\Temp\ipykernel_378520\970366390.py:88: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref_usd = ref_concat.groupby("date").apply(lambda g: pd.Series({
C:\Users\Pedro\AppData\Local\Temp\ipykernel_378520\970366390.py:92: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  ref_usd["timestamp"] = (ref_usd["date"].dt.tz_convert("UTC") if pd.api.types.is_datetime64tz_dtype(ref_usd["date"]) else pd.to_datetime(ref_usd["date"], utc=True))


=== Construyendo vela agregada (USD) ===
[AGG] outliers filtrados: 7 filas
[AGG]   - Binance: 3
[AGG]   - Bitget: 1
[AGG]   - Bybit: 1
[AGG]   - Coinbase: 1
[AGG]   - OKX: 1


...

                     Ganadores 30D (pp de cuota)                      
                                                                      
  name               share_change_pp   slope_pp_per_30d   avg_weight  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Binance BTC/USDT             +3.05              +3.08       19.27%  
  OKX BTC/USDT                 +1.41              +1.61        7.82%  
  Binance BTC/USDC             +0.65              +0.77        4.71% 

                     Perdedores 30D (pp de cuota)                     
                                                                      
  name               share_change_pp   slope_pp_per_30d   avg_weight  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Bitget BTC/USDT              -3.61              -7.39        5.37%  
  Bybit BTC/USDT               -1.69              -0.65        8.48%  
  Coinbase BTC/USD             -0.55              -0.04        6.24% 

                       Ganadores 90D (pp de cuota)                       
                                                                         
  name                  share_change_pp   slope_pp_per_30d   avg_weight  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  MEXC BTC/USDT                   +7.89              +3.61       18.00%  
  Crypto.com BTC/USDT             +0.70              -0.14        5.35%  
  OKX BTC/USDT                    +0.12              +0.45        7.07% 

                      Perdedores 90D (pp de cuota)                       
                                                                         
  name                  share_change_pp   slope_pp_per_30d   avg_weight  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Hyperliquid BTC/USD             -3.76              -1.78       22.52%  
  Bybit BTC/USDT                  -1.32              +0.02        9.04%  
  Bitget BTC/USDT                 -0.75              -2.07        7.94% 

 - Tablas (HTML): exchange_charts\winners_losers.html
=== Creando todas las visualizaciones ===

=== Creating Aggregated Chart with Volume ===
  ✓ Saved aggregated chart → exchange_charts\btc_aggregated_with_volume.html

=== Creating Individual Pair Charts ===
  ✓ Created chart for Binance BTC/USDT → binance_btc_usdt_chart.html
  ✓ Created chart for Binance BTC/USDC → binance_btc_usdc_chart.html
  ✓ Created chart for Bybit BTC/USDT → bybit_btc_usdt_chart.html
  ✓ Created chart for Bybit BTC/USDC → bybit_btc_usdc_chart.html
  ✓ Created chart for OKX BTC/USDT → okx_btc_usdt_chart.html
  ✓ Created chart for OKX BTC/USDC → okx_btc_usdc_chart.html
  ✓ Created chart for Coinbase BTC/USD → coinbase_btc_usd_chart.html
  ✓ Created chart for Crypto.com BTC/USDT → crypto.com_btc_usdt_chart.html
  ✓ Created chart for Crypto.com BTC/USDC → crypto.com_btc_usdc_chart.html
  ✓ Created chart for Hyperliquid BTC/USDC → hyperliquid_btc_usdc_chart.html
  ✓ Created chart for Upbit BTC/KRW → upbit_btc_krw_c